In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\AB_NYC_2019.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df.reviews_per_month.isnull().sum()

10052

In [82]:
#1. 전처리
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [83]:
#2. X, y 나누기
X = df.drop(columns = ['id', 'name', 'host_id', 'host_name',  'last_review', 'price', 'reviews_per_month'])
y = df.price

In [70]:
X.info() # neighbourhood_group, neighbourhood, room_type 라벨인코딩 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   neighbourhood                   48895 non-null  object 
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   room_type                       48895 non-null  object 
 5   minimum_nights                  48895 non-null  int64  
 6   number_of_reviews               48895 non-null  int64  
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 3.4+ MB


In [79]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])

In [84]:
# X label encoding
from sklearn.preprocessing import LabelEncoder
cols = ['neighbourhood_group', 'neighbourhood', 'room_type']
le = LabelEncoder()
X[cols] = X[cols].apply(lambda x: le.fit_transform(x))

In [25]:
# onehot encoding
# X = pd.get_dummies(X, columns = ['room_type'])

In [99]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  int32  
 1   neighbourhood                   48895 non-null  int32  
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   room_type                       48895 non-null  int32  
 5   minimum_nights                  48895 non-null  int64  
 6   number_of_reviews               48895 non-null  int64  
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
dtypes: float64(2), int32(3), int64(4)
memory usage: 2.8 MB


In [104]:
# 위경도 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X[['latitude', 'longitude']])

In [105]:
X_scale

array([[0.35739347, 0.51192067],
       [0.61419895, 0.49046911],
       [0.74825175, 0.56925653],
       ...,
       [0.76211678, 0.55651732],
       [0.62361168, 0.4766385 ],
       [0.63941249, 0.48000677]])

In [106]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2024)

In [107]:
#4. 모델학습
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [108]:
# 5. 예측
pred = lr.predict(X_test)

In [109]:
# 6. 모델평가
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(pred, y_test)
mae = mean_absolute_error(pred, y_test)
r2s = r2_score(pred, y_test)

print('MSE : ', mse)
print('RMSE : ', np.sqrt(mse))
print('MAE : ', mae)
print('r2 : ', r2s)

MSE :  67372.90194788222
RMSE :  259.5629055698873
MAE :  77.28395167222345
r2 :  -12.005340924506722


In [97]:
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)

In [98]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(pred, y_test)
mae = mean_absolute_error(pred, y_test)
r2s = r2_score(pred, y_test)

print('MSE : ', mse)
print('RMSE : ', np.sqrt(mse))
print('MAE : ', mae)
print('r2 : ', r2s)

MSE :  66108.74857787958
RMSE :  257.1162160928003
MAE :  69.66980857329843
r2 :  -2.229327058831929
